In [ ]:
import tensorflow as tf
import cv2
import os
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip drive/MyDrive/combined_faces.zip

Streaming output truncated to the last 5000 lines.
 extracting: content/combined_faces/1_1574.jpg  
 extracting: content/combined_faces/26_1801.jpg  
 extracting: content/combined_faces/3_306.jpg  
 extracting: content/combined_faces/2_632.jpg  
 extracting: content/combined_faces/26_439.jpg  
 extracting: content/combined_faces/35_868.jpg  
 extracting: content/combined_faces/45_418.jpg  
 extracting: content/combined_faces/24_792.jpg  
 extracting: content/combined_faces/35_697.jpg  
 extracting: content/combined_faces/24_928.jpg  
 extracting: content/combined_faces/78_69.jpg  
 extracting: content/combined_faces/12_31.jpg  
 extracting: content/combined_faces/60_117.jpg  
 extracting: content/combined_faces/27_323.jpg  
 extracting: content/combined_faces/1_613.jpg  
 extracting: content/combined_faces/52_260.jpg  
 extracting: content/combined_faces/27_700.jpg  
 extracting: content/combined_faces/59_79.jpg  
 extracting: content/combined_faces/3_190.jpg  
 extracting: content/com

In [ ]:
data = os.listdir('content/combined_faces/')

In [ ]:
classes = ['1-2','3-9','10-20','21-27','28-45','46-65','66-116']
X = []
y = []
l = len(data)
for i in range(l):
    X.append(cv2.imread(f'content/combined_faces/{classes[i]}',0))
    age = int(classes[i].split('_')[0])
    
    if age>=1 and age<=2:
        y.append(0)
    elif age>=3 and age<=9:
        y.append(1)
    elif age>=10 and age<=20:
        y.append(2)
    elif age>=21 and age<=27:
        y.append(3)
    elif age>=28 and age<=45:
        y.append(4)
    elif age>=46 and age<=65:
        y.append(5)
    elif age>=66 and age<=116:
        y.append(6)
    
np.savez_compressed('compressed image data.npz',x=X,y=y)

In [ ]:
loaded = np.load('compressed image data.npz',allow_pickle = True)

In [ ]:
X = loaded['x']

In [ ]:
y = loaded['y']

In [ ]:
y = np_utils.to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train = np.array(X_train).reshape(-1,200,200,1)
X_test = np.array(X_test).reshape(-1,200,200,1)

height = 200
width = 200
size = (height,width)
batch_size = 128
epochs = 60

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1,
                                   rotation_range=15)
test_datagen = ImageDataGenerator(rescale=1./255)
train_data = train_datagen.flow(X_train,y_train,batch_size)
test_data = test_datagen.flow(X_test,y_test,batch_size)

In [ ]:
final_cnn = Sequential()
final_cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(200, 200, 1)))   
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(GlobalAveragePooling2D())
final_cnn.add(Dense(132, activation='relu'))
final_cnn.add(Dense(7, activation='softmax'))
final_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:

checkpoint = ModelCheckpoint(filepath="final_cnn_model_checkpoint.h5",
                             monitor='val_accuracy',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )

In [ ]:
history = final_cnn.fit(train_data,
                    batch_size=batch_size,
                    validation_data=test_data,
                    epochs=epochs,
                    callbacks=[checkpoint],
                    shuffle=False    
                    ) 

Epoch 1/60
184/184 [==============================] - ETA: 0s - loss: 1.7970 - accuracy: 0.2876
Epoch 1: val_accuracy improved from -inf to 0.32321, saving model to final_cnn_model_checkpoint.h5
184/184 [==============================] - 93s 420ms/step - loss: 1.7970 - accuracy: 0.2876 - val_loss: 1.7716 - val_accuracy: 0.3232
Epoch 2/60
183/184 [============================>.] - ETA: 0s - loss: 1.7373 - accuracy: 0.3137
Epoch 2: val_accuracy improved from 0.32321 to 0.32580, saving model to final_cnn_model_checkpoint.h5
184/184 [==============================] - 76s 413ms/step - loss: 1.7371 - accuracy: 0.3138 - val_loss: 1.7079 - val_accuracy: 0.3258
Epoch 3/60
183/184 [============================>.] - ETA: 0s - loss: 1.7113 - accuracy: 0.3267
Epoch 3: val_accuracy improved from 0.32580 to 0.34939, saving model to final_cnn_model_checkpoint.h5
184/184 [==============================] - 76s 412ms/step - loss: 1.7112 - accuracy: 0.3267 - val_loss: 1.6645 - val_accuracy: 0.3494
Epoch 4

In [ ]:
from google.colab import files
files.download('final_cnn_model_checkpoint.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>